### Import Functions

In [1]:
#General Load and Save Functions
import numpy as np
import pandas as pd
from pandas import read_csv
import pickle
import glob
import time
import csv
import os
import statistics
from math import sqrt
import io
import statsmodels.api as sm

#Functions for loading and cleaning the Web Data
import bs4 as bs
import urllib
import urllib.request
import re
import requests

#SKlearn packages for pre-processing the train and test data
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#XGBoost
try:
    import xgboost as xgb
    from xgboost import XGBRegressor
except:
    print("Can't import XGBoost")
    pass

#Neural Network Packages
try:
    import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasRegressor
    from keras.layers import Dropout
except:
    print("Can't import Neural Network Modules")
    pass

try:
    from pulp import *
except:
    print("Can't import pulp")
    pass

#Data Viz
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", None, "display.max_columns", None)

def Grouper(df,column,criteria):
    column = column
    grouped = df.groupby(df[column])
    g = grouped.get_group(criteria)
    return g

#ResultsDB
#https://rotogrinders.com/resultsdb/site/draftkings/date/2020-07-29/sport/mlb/slate/5f22268c9985736e995ae6ad

print("Import Complete")

Can't import Neural Network Modules
Import Complete


### Load Master DataFrame

In [4]:
def Load_Master_DF(file):
    df = pd.read_csv(file)
    return df


MDF = Load_Master_DF('data/Submission/MLFiends.csv')
MDF['Team'] = MDF['TeamAbbrev']
MDF['UniquePlayer'] = MDF['ID']

print(MDF.shape)
print(MDF.head())
print("MDF Loaded")

(202, 40)
   Unnamed: 0  Unnamed: 0.1 Position                       Name + ID  \
0           0             0       RB  Christian McCaffrey (15783338)   
1           1             1       RB         Alvin Kamara (15783340)   
2           2             2       RB          Dalvin Cook (15783342)   
3           3             3       WR        Davante Adams (15783560)   
4           4             4       RB        Derrick Henry (15783344)   

                  Name        ID Roster Position  Salary  \
0  Christian McCaffrey  15783338         RB/FLEX    9400   
1         Alvin Kamara  15783340         RB/FLEX    9200   
2          Dalvin Cook  15783342         RB/FLEX    9000   
3        Davante Adams  15783560         WR/FLEX    8600   
4        Derrick Henry  15783344         RB/FLEX    8000   

                       Game Info  TeamAbbrev  AvgPointsPerGame  UniquePlayer  \
0  DET@CAR 11/22/2020 01:00PM ET    15260878             30.13      15783338   
1   ATL@NO 11/22/2020 01:00PM ET    

### Customized Dataframes

In [5]:
attlist = ['UniquePlayer','Position','Week','Salary','OurProjection','Team']
dk = pd.read_csv("data/2020/draftkings/Week6.csv")
Def2020 = pd.read_csv('data/2020/DefenseMatch.csv')
dkdef = Grouper(dk,'Position','DST').replace(dict(zip(Def2020.TeamAbbrev, Def2020.ID)))
dkdef['Team'] = dkdef['TeamAbbrev']
dkdef['Week'] = 6

projdef = pd.read_csv('data/2020/projections/Week6/FantasyPros_Fantasy_Football_Projections_DST.csv')
projdef['Team'] = projdef['Player']
projdef = projdef.dropna()
projdef['Player'] = projdef['Player'].replace({'New York Giants':'15260879', 
                                               'Pittsburgh Steelers':'15260860', 
                                               'Jacksonville Jaguars':'15260884', 
                                               'Denver Broncos':'15260873', 
                                               'Chicago Bears':'15260864', 
                                               'Tennessee Titans':'15260871', 
                                               'Minnesota Vikings':'15260877', 
                                               'Atlanta Falcons':'15260883', 
                                               'Tampa Bay Buccaneers':'15260886', 
                                               'Dallas Cowboys':'15260870', 
                                               'Kansas City Chiefs':'15260868', 
                                               'Los Angeles Rams':'15260882',
                                               'Los Angeles Chargers':'15260872', 
                                               'Cincinnati Bengals':'15260869', 
                                               'Philadelphia Eagles':'15260862', 
                                               'Green Bay Packers':'15260875', 
                                               'Washington Football Team':'15260891', 
                                               'Carolina Panthers':'15260878', 
                                               'Detroit Lions':'15260874', 
                                               'New Orleans Saints':'15260880', 
                                               'New England Patriots':'15260865', 
                                               'Buffalo Bills':'15260861', 
                                               'Baltimore Ravens':'15260866', 
                                               'San Francisco 49ers':'15260863', 
                                               'Indianapolis Colts':'15260867', 
                                               'Seattle Seahawks':'15260881',
                                               'Arizona Cardinals':'15260888', 
                                               'Cleveland Browns':'15260885', 
                                               'Houston Texans':'15260889', 
                                               'Las Vegas Raiders':'15260876', 
                                               'Miami Dolphins':'15260890', 
                                               'New York Jets':'15260887'})

projdef['Team'] = projdef['Player'].astype({'Player': 'int64'})
projdef['OurProjection'] = projdef['FPTS']


defense = dkdef.merge(projdef, on=['Team'])
defense['UniquePlayer'] = defense['ID']


def add_defense(week):
    for file in glob.glob(r'data\2020\draftkings\*'):
        w = file.split("\\")[-1].split(".")[0].replace("Week","")
        if w == str(week):
            with open(file) as f:
                reader = csv.reader(f)
                dk = [row for row in reader]
                headers = dk.pop(0)
                DK = pd.DataFrame(dk,columns=headers)
                df = Grouper(DK,'Position','DST')
                selectdef = df[df.Salary == df.Salary.min()]
                salary = int(selectdef['Salary'])
                ID = int(selectdef['ID'])
    return ID, salary

print(add_defense('10'))


(15758136, 2000)


In [6]:
def OptimizeRoster(data,week):
    attlist = ['UniquePlayer','Position','Week','Salary','OurProjection','Team']
    defID, defSalary = add_defense(week)
    sal_cap = 50000 - defSalary
    df_ori = data[attlist].fillna(0)
    df_ori = add_defence(df_ori,defense,attlist)
    df = df_ori.copy()
    df = Grouper(df,'Week',week)
    df = df[df['Salary']!=0]
    df['num'] = 1
    salaries = {}
    points = {}
    nums = {}
    for pos in df['Position'].unique():
        available_pos = df[df.Position == pos]
        salary = list(available_pos[["UniquePlayer","Salary"]].set_index("UniquePlayer").to_dict().values())[0]
        point = list(available_pos[["UniquePlayer","OurProjection"]].set_index("UniquePlayer").to_dict().values())[0]
        num = list(available_pos[["UniquePlayer","num"]].set_index("UniquePlayer").to_dict().values())[0]
        salaries[pos] = salary
        points[pos] = point
        nums[pos] = num
    pos_num_available = {
        "QB": 1,
        "RB": 3,
        "WR": 4,
        "TE": 2,
        "DST":1
    }
    _vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}
    prob = LpProblem("Fantasy_Optimization", LpMaximize)
    rewards = []
    costs = []
    total_num = []

    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        total_num += lpSum([nums[k][i] * _vars[k][i] for i in v])
        if k in ['RB','WR','TE']:
            prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
            prob += lpSum([_vars[k][i] for i in v]) >= pos_num_available[k]-1
        else:
            prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]
    
    prob += lpSum(total_num) <= 9
    prob += lpSum(rewards)
    prob += lpSum(costs) <= sal_cap
    prob.solve()
    lineup_df = pd.DataFrame(columns=['Position','UniquePlayer'])
    for v in prob.variables():
        if v.varValue>0:
            pos_name = v.name.replace("_", " ", 1).split(" ")
            lineup_df = lineup_df.append({'Position':(pos_name[0]),'UniquePlayer':(pos_name[1].replace("_","-"))},ignore_index=True)
    pos_num = lineup_df['Position'].value_counts().to_frame()
    if pos_num.loc['RB'].values.item() == 3:
        lineup_df.at[lineup_df['Position'].tolist().index('RB'),'Position']='FLEX'
    elif pos_num.loc['WR'].values.item() == 4:
        lineup_df.at[lineup_df['Position'].tolist().index('WR'),'Position']='FLEX'
    elif pos_num.loc['TE'].values.item() == 2:
        lineup_df.at[lineup_df['Position'].tolist().index('TE'),'Position']='FLEX'
        
    ### This is needed if you want to do the matching in the code
    #df = df.drop(['Position'], axis=1)
    #lineup_df['UniquePlayer'] = lineup_df['UniquePlayer'].astype({'UniquePlayer':'int64'})
    #df['UniquePlayer'] = df['UniquePlayer'].astype({'UniquePlayer':'int64'})
    #DK_lineup_df = lineup_df.merge(df,left_on=['UniquePlayer'],right_on=['UniquePlayer'],how='left')
    #DK_lineup_df = DK_lineup_df[['Position','UniquePlayer','Salary','OurProjection']]
    lineup_df = lineup_df.append({'Position':'DST','UniquePlayer':str(defID)},ignore_index=True)
    return lineup_df

print(OptimizeRoster(MDF,11))

TypeError: cannot convert the series to <class 'int'>

In [ ]:
lineup_df = OptimizeRoster(MDF,11)
PosList = lineup_df['Position'].tolist()
PlayerList = lineup_df['UniquePlayer'].tolist()
TeamName = 'MLFiends'
fname = TeamName + 'Roster.csv'

with open('data/Submission/'+fname, 'w', newline='') as csvfile:
    rwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    rwriter.writerow(PosList)
    rwriter.writerow(PlayerList)